# Panel Plots
We are creating this notebook in order to practice creating panel plots
for Savitzky smoothing method and see how different parameters affect our estimates.

In [1]:
import csv
import numpy as np
import pandas as pd
# import geopandas as gpd
from IPython.display import Image
# from shapely.geometry import Point, Polygon
from math import factorial
import datetime
import time
import scipy
import scipy.signal
import os, os.path

from statsmodels.sandbox.regression.predstd import wls_prediction_std
from sklearn.linear_model import LinearRegression
from patsy import cr

# from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sb


import sys
start_time = time.time()

In [5]:
# search path for modules
# look @ https://stackoverflow.com/questions/67631/how-to-import-a-module-given-the-full-path


####################################################################################
###
###                      Local
###
####################################################################################

################
###
### Core path
###

sys.path.append('/Users/hn/Documents/00_GitHub/Ag/remote_sensing/python/')

################
###
### Directories
###
data_dir = "/Users/hn/Documents/01_research_data" + \
           "/remote_sensing/01_NDVI_TS/00_Eastern_WA_withYear/"

param_dir = "/Users/hn/Documents/00_GitHub/Ag/remote_sensing/parameters/"
####################################################################################
###
###                      Aeolus Core path
###
####################################################################################

# sys.path.append('/home/hnoorazar/remote_sensing_codes/')

####################################################################################
###
###                   Aeolus Directories
###
####################################################################################

# data_dir = "/data/hydro/users/Hossein/remote_sensing/01_NDVI_TS/00_Eastern_WA_EE/"
# param_dir = "/home/hnoorazar/remote_sensing_codes/parameters/"

####################################################################################
###
###                   Import remote cores
###
####################################################################################

import remote_sensing_core as rc
import remote_sensing_core as rcp


In [6]:
####################################################################################
###
###      Parameters                   
###
####################################################################################
freedom_df = 7

Sav_win_size = 9
sav_order = 1
delt = 0.1
indeks = "EVI"
irrigated_only = 0

# Sav_win_size = int(sys.argv[1])
# sav_order = int(sys.argv[2])
# delt = float(sys.argv[3])
# indeks = sys.argv[4]
# irrigated_only = sys.argv[5]
irrigated_only = 0

print ("delta = {fileShape}.".format(fileShape=delt))


delta = 0.1.


In [40]:
####################################################################################
###
###                   process data
###
####################################################################################
f_name = "Eastern_WA_2017_70cloud_selectors.csv"
f_name = "Grant2017_4_Kirti_plot_TS_70cloud_selectors.csv"
a_df = pd.read_csv(data_dir + f_name, low_memory=False)


##################################################################
##################################################################
####
####  plots has to be exact. So, we need 
#### to filter out NASS, and filter by last survey date
####
##################################################################
##################################################################

a_df = a_df[a_df['county']== "Grant"] # Filter Grant
a_df = rc.filter_out_NASS(a_df) # Toss NASS
a_df = rc.filter_by_lastSurvey(a_df, year = 2017) # filter by last survey date
a_df['year'] = 2017

if irrigated_only == True:
    filter_out_nonIrrigated = filter_out_nonIrrigated(filter_out_nonIrrigated)
    output_Irr = "irrigated"
else:
    output_Irr = "non_irrigated"

In [ ]:
output_dir = "/data/hydro/users/Hossein/remote_sensing/01_NDVI_TS/01_Eastern_WA_plots_tbls/plots/Grant_2017/" + \
             output_Irr + "/savitzky_" + indeks + "/" + \
             "/delta" + str(delt) + "_Sav_win" + str(Sav_win_size) + "_Order"  + str(sav_order) + "/"

plot_dir_base = output_dir
print ("plot_dir_base is " + plot_dir_base)

os.makedirs(output_dir, exist_ok=True)
os.makedirs(plot_dir_base, exist_ok=True)

In [47]:
######################

# The following columns do not exist in the old data
#
if not('DataSrc' in a_df.columns):
    print ("Data source is being set to NA")
    a_df['DataSrc'] = "NA"

if not('CovrCrp' in a_df.columns):
    print ("Data source is being set to NA")
    a_df['CovrCrp'] = "NA"

if (indeks == "EVI"):
    a_df = rc.initial_clean_EVI(a_df)
else:
    a_df = rc.initial_clean_NDVI(a_df)

a_df.head(2)
an_EE_TS = a_df.copy()

Data source is being set to NA


In [50]:
### List of unique polygons
polygon_list = an_EE_TS['ID'].unique()
print(len(polygon_list))

1768
